# Introduction

## Background

Barcelona is one of the world's leading tourist, economic, trade fair and cultural centres, and its influence in commerce, education, entertainment, sports, media, fashion, science, and the arts all contribute to its status as one of the major global cities.
Barcelona is also one of the most densely populated cities in Europe with 1,636.762 people in 2019, a surface area of 101.35 km2 giving the city an average population density of 16,150 inhabitants per square kilometre with Eixample being the most populated district.
Since 1987, the city has been divided into 10 administrative districts: 
- Ciutat Vella 
- Eixample
- Sants-Montjuïc
- Les Corts
- Sarrià-Sant Gervasi
- Gràcia
- Horta-Guinardó
- Nou Barris
- Sant Andreu
- Sant Martí


## Main Goal

Due to crisis of COVID-19 all the cities around the world have reported a huge impact in a lot of aspects but what is happening in the real state market? 
Banks do not understand of crisis and want to keep mortgages' payments up to date. One of the most relevant indicators (at least in Spain) about the situation is the real state pricing trend.
Based on intuition, seems to be reasonable believing that in this critical situation the real state market should change the trend or at least should show some changes based on it.

Barcelona is divided into 10 districts. Obviously districts are very different as the people who lives in them. The current behaviour of real-state market should be different depending on purchasing power.

The main goal is finding an accurate prediction about real state market situation for next months for each district and demonstrating the impact of the COVID-19 crisis.
Let's analyse the consequences of COVID-19 in the real state market in Barcelona. 

# Project Code

__Importing required libraries__

In [50]:
#Pandas
import pandas as pd
import numpy as np
from pandas.tseries.offsets import DateOffset

#Files
from pathlib import Path

#Graphics
import matplotlib.pyplot as plt 
import plotly.graph_objects as go
import plotly.express as px

#Deep learning
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from statsmodels.tools.eval_measures import rmse

#Remote modules
#%run graphics.ipynb
#%run cnn.ipynb

__Convert CSV files in dataframes__

In [51]:
df_selling = pd.read_csv('Barcelona_selling_prices.csv') 

In [52]:
print (df_selling.shape)
df_selling.head()

(1720, 4)


,district_code,district_name,date,selling_price_m2
0,1,Ciutat Vella,2006-01,n.d.
1,1,Ciutat Vella,2006-02,n.d.
2,1,Ciutat Vella,2006-03,n.d.
3,1,Ciutat Vella,2006-04,n.d.
4,1,Ciutat Vella,2006-05,n.d.


__Delete Empty Rows ("n.d.)__

In [53]:
df_selling = df_selling[df_selling.selling_price_m2 != 'n.d.']
df_selling.head()

,district_code,district_name,date,selling_price_m2
33,1,Ciutat Vella,2008-10,4046
34,1,Ciutat Vella,2008-11,4029
35,1,Ciutat Vella,2008-12,3959
36,1,Ciutat Vella,2009-01,4029
37,1,Ciutat Vella,2009-02,4138


In [54]:
print ("\ndf_selling data types:\n" + str(df_selling.dtypes))


df_selling data types:
district_code        int64
district_name       object
date                object
selling_price_m2    object
dtype: object


In [55]:
df_selling['date'] = pd.to_datetime(df_selling['date'])
df_selling['selling_price_m2'] = pd.to_numeric(df_selling['selling_price_m2'])
df_selling.head()

,district_code,district_name,date,selling_price_m2
33,1,Ciutat Vella,2008-10-01,4046
34,1,Ciutat Vella,2008-11-01,4029
35,1,Ciutat Vella,2008-12-01,3959
36,1,Ciutat Vella,2009-01-01,4029
37,1,Ciutat Vella,2009-02-01,4138


__Formatting data types__

In [56]:
print ("\n df_selling shape:" + str(df_selling.shape))
print ("\ndf_selling data types:\n" + str(df_selling.dtypes))
df_selling.head()


 df_selling shape:(1354, 4)

df_selling data types:
district_code                int64
district_name               object
date                datetime64[ns]
selling_price_m2             int64
dtype: object


,district_code,district_name,date,selling_price_m2
33,1,Ciutat Vella,2008-10-01,4046
34,1,Ciutat Vella,2008-11-01,4029
35,1,Ciutat Vella,2008-12-01,3959
36,1,Ciutat Vella,2009-01-01,4029
37,1,Ciutat Vella,2009-02-01,4138


__Global Pricing distribution (all districts)__

In [57]:
fig = px.histogram(df_selling,'selling_price_m2',nbins=30)
fig.show()

__Districts Analysis__

In [58]:
df_districts = df_selling.copy()
df_districts = df_districts.drop(['date','selling_price_m2'],axis=1)
df_districts = df_districts.drop_duplicates()
df_districts.head(10)

,district_code,district_name
33,1,Ciutat Vella
201,2,Eixample
379,3,Sants-Montjuic
563,4,Les Corts
729,5,Sarrià – Sant Gervasi
902,6,Gràcia
1064,7,Horta-Guinardó
1241,8,Nou Barris
1415,9,Sant Andreu
1578,10,Sant Martí


__Global Plot__

In [59]:
fig = go.Figure()

for i in range (1,11):
    df = df_selling[df_selling.district_code == i].groupby(['date'], as_index=False).mean()
    fig.add_trace(go.Scatter(
        x=df['date'],
        y=df['selling_price_m2'],
        name=df_districts.iloc[i-1,1]       # this sets its legend entry
    ))

fig.update_layout(
    title="Real estate market pricing in Barcelona (by District)",
    yaxis_title="€/m2",
    width=900, height=500,
    font=dict(
        family="Arial, monospace",
        size=12,
        color="#7f7f7f"
    )
)
fig.show()

__Constants definition for district's name__

In [60]:
CIUTATVELLA = 1
EIXAMPLE = 2
SANTS = 3
CORTS = 4
SARRIA = 5
GRACIA = 6
HORTA = 7
NOUBARRIS = 8
SANTANDREU = 9
SANTMARTI = 10

__Input box in order to get district for calculation__

In [61]:
while True:
  num = input("Please enter a valid district code (see lines above?)")
  try:
    val = int(num)
    if (val >= 1 and val <=10):
        print("Setting current district to : " + df_districts.iloc[val-1,1].upper())
        CURRENT_DISTRICT = val
        break;
  except ValueError:
      print ("This is not a valid district code. Try again")

Please enter a valid district code (see lines above?)6
Setting current district to : GRÀCIA


__Real estate marker pricing by distric__

In [62]:
district_name = df_districts.iloc[CURRENT_DISTRICT-1,1]
title = "Real State market pricing for district: "  + district_name
df_graph =  df_selling[df_selling.district_code == CURRENT_DISTRICT].groupby(['date'], as_index=False).mean()

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_graph['date'],
    y=df_graph['selling_price_m2'],
    name = district_name
))

fig.update_layout(
    title="Real State market pricing for district: "  + district_name,
    yaxis_title="€/m2",
    width=900, height=500,
    font=dict(
        family="Arial, monospace",
        size=12,
        color="#7f7f7f"
    )
)
fig.show()

__Dividing the rows in two windows (train - test method)__

I have divided the rows into two windows. The first window contains all the records until -6 and the second window contains the records from -6 until the end. With the information of the first window I predicted the information of the second window. With both windows we can compare the precision of the model using root mean squared error (rmse).

__Preparing train set & test set__

In [63]:
df = df_selling[df_selling.district_code == CURRENT_DISTRICT].drop(['district_code','district_name'], axis=1)
df = df.set_index("date")
train = df[:-6] #All the records from start to end (except las 6 rows for prediction)
test = df[-6:] #Las 6 records

__Scaling information for optimize calculations__

In [64]:
scaler = MinMaxScaler()
scaler.fit(train)
train = scaler.transform(train)
test = scaler.transform(test)

__Time Series Generator__

Creating 6 aditional dates after the train set dates in order to get predictions.

In [65]:
n_input = 6
n_features = 1

In [66]:
generator = TimeseriesGenerator(train, train, length=n_input, batch_size=12)

__Model Creation__

I tried with some combinations but the best results I got were with this...

In [67]:
model = Sequential()
model.add(LSTM(units=200, activation='relu', return_sequences=True, input_shape=(n_input, n_features)))    
model.add(Dropout(0.2))
model.add(LSTM(units=200))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(optimizer = 'sgd', loss = 'mse', metrics=['mse', 'mae','accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 6, 200)            161600    
_________________________________________________________________
dropout_3 (Dropout)          (None, 6, 200)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 200)               320800    
_________________________________________________________________
dropout_4 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 201       
Total params: 482,601
Trainable params: 482,601
Non-trainable params: 0
_________________________________________________________________


__TIP : Useful function__

With early_stopper_train function we can detect how calculation is running (epoch by epoch) and avoid overfitting. Once the calculation is optimal this parameter stop the process saving memory and time

In [68]:
early_stopper = EarlyStopping(monitor='loss', min_delta=0, patience=20, verbose=1, mode='auto', baseline=None, restore_best_weights=False)

__Fitting the Model__

In [69]:
result = model.fit_generator(generator,epochs=100, callbacks=[early_stopper])

Epoch 1/100
10/10 [==============================] - 4s 396ms/step - loss: 0.2187 - mse: 0.2117 - mae: 0.3452 - accuracy: 0.0085
Epoch 2/100
10/10 [==============================] - 0s 49ms/step - loss: 0.1393 - mse: 0.1361 - mae: 0.2926 - accuracy: 0.0085
Epoch 3/100
10/10 [==============================] - 1s 53ms/step - loss: 0.1111 - mse: 0.1085 - mae: 0.2837 - accuracy: 0.0085
Epoch 4/100
10/10 [==============================] - 0s 48ms/step - loss: 0.1008 - mse: 0.0989 - mae: 0.2856 - accuracy: 0.0085
Epoch 5/100
10/10 [==============================] - 1s 50ms/step - loss: 0.0990 - mse: 0.0972 - mae: 0.2805 - accuracy: 0.0085
Epoch 6/100
10/10 [==============================] - 0s 49ms/step - loss: 0.0947 - mse: 0.0933 - mae: 0.2791 - accuracy: 0.0085
Epoch 7/100
10/10 [==============================] - 1s 66ms/step - loss: 0.0919 - mse: 0.0906 - mae: 0.2722 - accuracy: 0.0085
Epoch 8/100
10/10 [==============================] - 1s 56ms/step - loss: 0.0916 - mse: 0.0903 - mae: 0

10/10 [==============================] - 0s 47ms/step - loss: 0.0198 - mse: 0.0198 - mae: 0.1273 - accuracy: 0.0169
Epoch 65/100
10/10 [==============================] - 0s 50ms/step - loss: 0.0200 - mse: 0.0199 - mae: 0.1249 - accuracy: 0.0169
Epoch 66/100
10/10 [==============================] - 0s 48ms/step - loss: 0.0182 - mse: 0.0182 - mae: 0.1169 - accuracy: 0.0169
Epoch 67/100
10/10 [==============================] - 1s 52ms/step - loss: 0.0184 - mse: 0.0185 - mae: 0.1189 - accuracy: 0.0169
Epoch 68/100
10/10 [==============================] - 0s 48ms/step - loss: 0.0172 - mse: 0.0171 - mae: 0.1182 - accuracy: 0.0169
Epoch 69/100
10/10 [==============================] - 1s 53ms/step - loss: 0.0168 - mse: 0.0168 - mae: 0.1132 - accuracy: 0.0169
Epoch 70/100
10/10 [==============================] - 1s 51ms/step - loss: 0.0158 - mse: 0.0158 - mae: 0.1104 - accuracy: 0.0169
Epoch 71/100
10/10 [==============================] - 0s 47ms/step - loss: 0.0162 - mse: 0.0162 - mae: 0.1110 

__Visual Results for training set__

In [70]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    y=result.history['loss'],
    name="loss"       # this sets its legend entry
))

fig.add_trace(go.Scatter(
    y=result.history['mae'],
    name="mean absoulte error"       # this sets its legend entry
))

fig.update_layout(
    title="Training results: "  + district_name,
    yaxis_title="Value",
    width=900, height=500,
    font=dict(
        family="Arial, monospace",
        size=12,
        color="#7f7f7f"
    )
)
fig.show()

__Creating a little dataset with both columms : Current values and predicted values__

In [71]:
pred_list = []

batch = train[-n_input:].reshape((1, n_input, n_features))

for i in range(n_input):   
    pred_list.append(model.predict(batch)[0]) 
    batch = np.append(batch[:,1:,:],[[pred_list[i]]],axis=1)

In [72]:
df_predict = pd.DataFrame(scaler.inverse_transform(pred_list),
                          index=df[-n_input:].index, columns=['prediction'])

df_test = pd.concat([df,df_predict], axis=1)

In [73]:
df_test.tail(6)

,selling_price_m2,prediction
date,,
2019-11-01,4286,4210.740096
2019-12-01,4217,4217.384563
2020-01-01,4176,4217.750585
2020-02-01,4185,4208.708154
2020-03-01,4185,4204.543451
2020-04-01,4193,4191.366592


In [74]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x = df_test.index,
    y=df_test['selling_price_m2'],
    name="selling_price_m2"       # this sets its legend entry
))

fig.add_trace(go.Scatter(
    x = df_test.index,
    y=df_test['prediction'],
    name="prediction"       # this sets its legend entry
))

fig.update_layout(
    title="Predicted vs. current values comparison for " + df_districts.iloc[CURRENT_DISTRICT-1,1],
    yaxis_title="selling price",
    width=900, height=500,
    font=dict(
        family="Arial, monospace",
        size=12,
        color="#7f7f7f"
    )
)
fig.show()

In [75]:
df_test.iloc[-n_input:, [0]]

,selling_price_m2
date,
2019-11-01,4286
2019-12-01,4217
2020-01-01,4176
2020-02-01,4185
2020-03-01,4185
2020-04-01,4193


In [76]:
df_test.iloc[-n_input:, [1]]

,prediction
date,
2019-11-01,4210.740096
2019-12-01,4217.384563
2020-01-01,4217.750585
2020-02-01,4208.708154
2020-03-01,4204.543451
2020-04-01,4191.366592


__Results__

__Root mean square error for training set__

The main goal is predict the last 6 months of the datasets based on the previous 6 months. We have predicted the selling price (€ /m2) for Nov 19 to April 20 2020 gathering the information from May 19 to Oct 19 2020. Our deviation in training set is around 31€ over the mean of current values for the same dates.

In [77]:
pred_actual_rmse = rmse(df_test.iloc[-n_input:, [0]], df_test.iloc[-n_input:, [1]])
mean_value = df_test.iloc[-n_input:, [0]].mean(axis=0)
print("Root mean squared error (€):",pred_actual_rmse[0].round(2))
error_per = (pred_actual_rmse * 100 / mean_value).round(2)
print("Error vs. reference value (%):", error_per.selling_price_m2)
print("Hit percentage (%): ", (100 - error_per.selling_price_m2).round(2))

Root mean squared error (€): 37.31
Error vs. reference value (%): 0.89
Hit percentage (%):  99.11


In [78]:
result = {
        'District_name': district_name ,
        'Prediction_type': 'training',
        'RMSE(€)': [pred_actual_rmse[0].round(2)],
        'Error(%)': [error_per.selling_price_m2],
        'Certainty(%)': [(100 - error_per.selling_price_m2).round(2)]
        }

df_result = pd.DataFrame(result, columns = ['District_name','Prediction_type','RMSE(€)','Error(%)','Certainty(%)'])
df_result.head()

,District_name,Prediction_type,RMSE(€),Error(%),Certainty(%)
0,Gràcia,training,37.31,0.89,99.11


__Predicting future values__

Then I'm triynig to predict the next six months based on the last six months values.

In [79]:
future = df

__Scaling information for optimising calculation__

In [80]:
scaler.fit(future)
future = scaler.transform(future)

__Creating 6 new records based on datetime month for future dates__

In [81]:
n_input = 6
n_features = 1
generator = TimeseriesGenerator(future, future, length=n_input, batch_size=12)

In [82]:
early_stopper_future = EarlyStopping(monitor='loss', min_delta=0, patience=20, verbose=1, mode='auto', baseline=None, restore_best_weights=False)

In [83]:
result2 = model.fit_generator(generator,epochs=200,callbacks = [early_stopper_future])

Epoch 1/200
11/11 [==============================] - 1s 71ms/step - loss: 0.0078 - mse: 0.0082 - mae: 0.0719 - accuracy: 0.0161
Epoch 2/200
11/11 [==============================] - 1s 89ms/step - loss: 0.0082 - mse: 0.0087 - mae: 0.0751 - accuracy: 0.0161
Epoch 3/200
11/11 [==============================] - 1s 65ms/step - loss: 0.0088 - mse: 0.0090 - mae: 0.0762 - accuracy: 0.0161
Epoch 4/200
11/11 [==============================] - 1s 55ms/step - loss: 0.0078 - mse: 0.0083 - mae: 0.0702 - accuracy: 0.0161
Epoch 5/200
11/11 [==============================] - 1s 52ms/step - loss: 0.0078 - mse: 0.0082 - mae: 0.0727 - accuracy: 0.0161
Epoch 6/200
11/11 [==============================] - 1s 60ms/step - loss: 0.0067 - mse: 0.0071 - mae: 0.0678 - accuracy: 0.0161
Epoch 7/200
11/11 [==============================] - 1s 55ms/step - loss: 0.0073 - mse: 0.0076 - mae: 0.0684 - accuracy: 0.0161
Epoch 8/200
11/11 [==============================] - 1s 53ms/step - loss: 0.0067 - mse: 0.0070 - mae: 0.

__Visual results for prediction set__

In [84]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    y=result2.history['loss'],
    name="loss"       # this sets its legend entry
))

fig.add_trace(go.Scatter(
    y=result2.history['mae'],
    name="mean absoulte error"       # this sets its legend entry
))

fig.update_layout(
    title="Prediction results error: "  + district_name,
    yaxis_title="Value",
    width=900, height=500,
    font=dict(
        family="Arial, monospace",
        size=12,
        color="#7f7f7f"
    )
)
fig.show()

In [85]:
pred_list = []

batch = train[-n_input:].reshape((1, n_input, n_features))

for i in range(n_input):   
    pred_list.append(model.predict(batch)[0]) 
    batch = np.append(batch[:,1:,:],[[pred_list[i]]],axis=1)

In [86]:
add_dates = [df.index[-1] + DateOffset(months=x) for x in range(0,7) ]
future_dates = pd.DataFrame(index=add_dates[1:],columns=df.columns)

In [87]:
future_dates

,selling_price_m2
2020-05-01,NaN
2020-06-01,NaN
2020-07-01,NaN
2020-08-01,NaN
2020-09-01,NaN
2020-10-01,NaN


__Creating new rows for future dates__

In [88]:
df_predict = pd.DataFrame(scaler.inverse_transform(pred_list),
                          index=future_dates[-n_input:].index, columns=['prediction'])

df_proj = pd.concat([df,df_predict], axis=1)

In [89]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x = df_proj.index,
    y = df_proj['selling_price_m2'],
    name="selling_price_m2"       # this sets its legend entry
))

fig.add_trace(go.Scatter(
    x = df_proj.index,
    y=df_proj['prediction'],
    name="prediction"       # this sets its legend entry
))

fig.update_layout(
    title="Future predicted values for " + df_districts.iloc[CURRENT_DISTRICT-1,1],
    yaxis_title="selling price",
    width=900, height=500,
    font=dict(
        family="Arial, monospace",
        size=12,
        color="#7f7f7f"
    )
)
fig.show()

__Previous 6 months__

In [90]:
df_proj.iloc[-12:-6,[0]]

,selling_price_m2
2019-11-01,4286.0
2019-12-01,4217.0
2020-01-01,4176.0
2020-02-01,4185.0
2020-03-01,4185.0
2020-04-01,4193.0


In [91]:
df_proj.iloc[-6:,[1]]

,prediction
2020-05-01,4234.606965
2020-06-01,4243.794756
2020-07-01,4247.693095
2020-08-01,4242.935014
2020-09-01,4244.524858
2020-10-01,4237.371822


__Prediction results__

In [92]:
pred_future_rmse = rmse(df_proj.iloc[-12:-6,[0]], df_proj.iloc[-6:,[1]])[0]
pred_future_rmse

53.80154257267215

In [93]:
mean_value = df_proj.iloc[-n_input:, [1]].mean(axis=0)
mean_value

prediction    4241.821085
dtype: float64

In [94]:
print("Root mean squared error (€):",pred_future_rmse.round(2))
error_per = (pred_future_rmse * 100 / mean_value).round(2)
print("Error vs. reference value (%):", error_per.prediction)
print("Hit percentage (%): ", (100 - error_per.prediction).round(2))

Root mean squared error (€): 53.8
Error vs. reference value (%): 1.27
Hit percentage (%):  98.73


In [95]:
new_row = {
        'District_name': district_name,
        'Prediction_type': 'prediction',
        'RMSE(€)': pred_future_rmse.round(2),
        'Error(%)': error_per.prediction,
        'Certainty(%)': (100 - error_per.prediction).round(2)
        }
df_result = df_result.append(new_row,ignore_index=True)

__Saving results to file__

In [96]:
filename = 'predictions_'+ df_districts.iloc[CURRENT_DISTRICT-1,1] + '.csv'
df_result.to_csv(filename,index=False)

__Reading all the stored results__

In [97]:
df_result = pd.DataFrame()

for i in range (1,11):
    filename = 'predictions_'+ df_districts.iloc[i-1,1] + '.csv'
    if Path(filename).is_file():
        print (filename)
        df_file = pd.read_csv(filename)
        df_result = pd.concat([df_result, df_file])
df_result.head(20)

predictions_Ciutat Vella.csv
predictions_Eixample.csv
predictions_Sants-Montjuic.csv
predictions_Les Corts.csv
predictions_Sarrià – Sant Gervasi.csv
predictions_Gràcia.csv
predictions_Horta-Guinardó.csv
predictions_Nou Barris.csv
predictions_Sant Andreu.csv
predictions_Sant Martí.csv


,District_name,Prediction_type,RMSE(€),Error(%),Certainty(%)
0,Ciutat Vella,training,33.04,0.76,99.24
1,Ciutat Vella,prediction,81.96,1.86,98.14
0,Eixample,training,86.70,1.85,98.15
1,Eixample,prediction,90.84,1.91,98.09
0,Sants-Montjuic,training,59.31,1.67,98.33
1,Sants-Montjuic,prediction,30.62,0.87,99.13
0,Les Corts,training,277.73,5.68,94.32
1,Les Corts,prediction,176.76,3.70,96.30
0,Sarrià – Sant Gervasi,train-test,31.76,0.60,99.40
1,Sarrià – Sant Gervasi,future,77.82,1.44,98.56


In [98]:
df_result[df_result.District_name == df_districts.iloc[CURRENT_DISTRICT-1,1]]

,District_name,Prediction_type,RMSE(€),Error(%),Certainty(%)
0,Gràcia,training,37.31,0.89,99.11
1,Gràcia,prediction,53.80,1.27,98.73
